In [1]:
import pandas as pd

train_data = pd.read_csv('./loan_data_track1.csv')

In [2]:
train_data.columns

Index(['checking_status', 'duration', 'loan_history', 'purpose', 'loan_amount',
       'savings_status', 'employment', 'installment_commitment',
       'personal_status', 'other_parties', 'residence_since',
       'property_magnitude', 'age', 'other_payment_plans', 'housing',
       'existing_loans', 'job', 'num_dependents', 'own_telephone',
       'foreign_worker', 'accepted'],
      dtype='object')

In [3]:
train_data.head()

,checking_status,duration,loan_history,purpose,loan_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_loans,job,num_dependents,own_telephone,foreign_worker,accepted
0,<0DM,6,Critical_acct_other_loans_existing,radio/television,1169,Unknown_or_no_savings_acct,>7yrs,4,male_single,None,...,real_estate,67,none,own,m22,skilled_employee/official,1,Yes,yes,2
1,0_to_200DM,48,Existing_loans_paid_till_now,radio/television,5951,<100DM,1_to_4yrs,2,female_divorced/separated/married,None,...,real_estate,22,none,own,e1,skilled_employee/official,1,No,yes,1
2,None,12,Critical_acct_other_loans_existing,education,2096,<100DM,4_to_7yrs,2,male_single,None,...,real_estate,49,none,own,1,unskilled_resident,2,No,yes,2
3,<0DM,42,Existing_loans_paid_till_now,furniture/equipment,7882,<100DM,4_to_7yrs,2,male_single,guarantor,...,building_society_savings_agreement/life_insurance,45,none,for_free,q1,skilled_employee/official,2,No,yes,2
4,<0DM,24,Delay_in_past,new_car,4870,<100DM,1_to_4yrs,3,male_single,None,...,unknown/no_property,53,none,for_free,p22,skilled_employee/official,2,No,yes,1


In [4]:
cols_dummy = [col for col in train_data.columns if train_data[col].dtype != 'int64']
cols_dummy

['checking_status',
 'loan_history',
 'purpose',
 'savings_status',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'existing_loans',
 'job',
 'own_telephone',
 'foreign_worker']

In [5]:
for col in train_data.columns:
    print ('{}: {} / {};'.format(col, len(train_data[col].unique()), len(train_data)))

checking_status: 4 / 1000;
duration: 33 / 1000;
loan_history: 5 / 1000;
purpose: 10 / 1000;
loan_amount: 921 / 1000;
savings_status: 5 / 1000;
employment: 5 / 1000;
installment_commitment: 4 / 1000;
personal_status: 4 / 1000;
other_parties: 3 / 1000;
residence_since: 4 / 1000;
property_magnitude: 4 / 1000;
age: 53 / 1000;
other_payment_plans: 3 / 1000;
housing: 3 / 1000;
existing_loans: 301 / 1000;
job: 4 / 1000;
num_dependents: 2 / 1000;
own_telephone: 2 / 1000;
foreign_worker: 2 / 1000;
accepted: 2 / 1000;


In [6]:
train_df = pd.get_dummies(train_data, columns=cols_dummy)

In [60]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

rf = RandomForestClassifier(n_estimators=20)

X = train_df.drop(['accepted'], axis=1).as_matrix()
y = train_df['accepted'].as_matrix()

#y_pred = cross_val_predict(rf, X, y, cv=5, method='predict')
y_pred_prob = [x[1] for x in cross_val_predict(rf, X, y, cv=5, method='predict_proba')]
y_pred = [2 if x > 0.5 else 1 for x in y_pred_prob]
y_pred_auc = [2 if x > 0.55 else 1 for x in y_pred_prob]
print (sum([1 if y_pred[i] == y[i] else 0 for i in range(len(y))]))
print (sum([1 if y_pred_auc[i] == y[i] else 0 for i in range(len(y))]))


758
747


In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=20)
y_pred = cross_val_predict(gbm, X, y, cv=5, method='predict')
sum([1 if y_pred[i] == y[i] else 0 for i in range(len(y))])

733

In [18]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

bag = BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=1.0, max_features=0.5)
y_pred = cross_val_predict(bag, X, y, cv=5, method='predict')
sum([1 if y_pred[i] == y[i] else 0 for i in range(len(y))])

738

In [65]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=100, solver='lbfgs')
y_pred = cross_val_predict(mlp, X, y, cv=5, method='predict')
sum([1 if y_pred[i] == y[i] else 0 for i in range(len(y))])

712

In [75]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import VotingClassifier

X = train_df.drop(['accepted'], axis=1).as_matrix()
y = train_df['accepted'].as_matrix()

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = MLPClassifier(hidden_layer_sizes=100, solver='lbfgs')
clf5 = BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=1.0, max_features=0.5)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), ('mlp', clf4), ('bagging', clf5)], voting='soft', weights=[3, 3, 1, 1, 2])

for clf, label in zip([clf1, clf2, clf3, clf4, clf5, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Multi-Layer Perceptron', 'Bagging_DT', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: {} (+/- {}) [{}]".format(scores.mean(), scores.std(), label))

Accuracy: 0.752 (+/- 0.017204650534085267) [Logistic Regression]
Accuracy: 0.719 (+/- 0.009695359714832668) [Random Forest]
Accuracy: 0.643 (+/- 0.031240998703626646) [naive Bayes]
Accuracy: 0.716 (+/- 0.016248076809271938) [Multi-Layer Perceptron]
Accuracy: 0.719 (+/- 0.01200000000000001) [Bagging_DT]
Accuracy: 0.739 (+/- 0.015620499351813323) [Ensemble]


In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

X = train_df.drop(['accepted'], axis=1).as_matrix()
y = train_df['accepted'].as_matrix()

clf1 = LogisticRegression(random_state=1)
scores_1 = cross_val_score(clf1, X, y, cv=5, scoring='accuracy')
print (scores_1.mean())

clf2 = RandomForestClassifier(random_state=1, n_estimators=30)
scores_2 = cross_val_score(clf2, X, y, cv=5, scoring='accuracy')
print (scores_2.mean())

0.752
0.757
